<img src="https://2.bp.blogspot.com/-066qpJs0Ttc/WiYPXGNYEYI/AAAAAAAAFu8/XbOaf7DqfDMM9truu3DkrkIGfRgP4zBzgCLcBGAs/s1600/udinus.jpg"  width="200">

# AMS - 11 - Document Similarity

oleh: Dr. Eng. Farrikh Alzami M.Kom; Abu Salam, M.Kom



# Rekomendasi Film dengan Kemiripan Dokumen

Sistem rekomendasi adalah salah satu aplikasi pembelajaran mesin yang populer dan paling banyak diadopsi. Mereka biasanya digunakan untuk merekomendasikan entitas kepada pengguna dan entitas ini dapat berupa apa saja seperti produk, film, layanan, dan sebagainya.

Contoh rekomendasi yang populer meliputi,
- Amazon menyarankan produk di situs webnya
- Amazon Prime, Netflix, Hotstar merekomendasikan film\acara
- YouTube merekomendasikan video untuk ditonton

Biasanya sistem pemberi rekomendasi dapat diimplementasikan dalam tiga cara:

- Simple Rule-based Recommenders / Rekomendasi Berbasis Aturan Sederhana: Biasanya berdasarkan metrik dan ambang batas global tertentu seperti popularitas film, peringkat global, dll.
- Content-based Recommenders / Rekomendasi berbasis konten: Ini didasarkan pada penyediaan entitas serupa berdasarkan entitas minat tertentu. Metadata konten dapat digunakan di sini seperti deskripsi film, genre, pemeran, sutradara, dan sebagainya
- Collaborative filtering Recommenders / Pemfilteran kolaboratif Rekomendasi: Di ​​sini kami tidak memerlukan metadata, tetapi kami mencoba memprediksi rekomendasi dan peringkat berdasarkan peringkat sebelumnya dari berbagai pengguna dan item tertentu.

Kami akan membangun sistem rekomendasi film di sini berdasarkan data\metadata yang berkaitan dengan film yang berbeda, kami mencoba dan merekomendasikan film serupa yang menarik!

![](https://i.imgur.com/c7Go7d3.png)

Karena fokus kami bukan pada mesin rekomendasi tetapi NLP, kami akan memanfaatkan metadata berbasis teks untuk setiap film untuk mencoba dan merekomendasikan film serupa berdasarkan film tertentu yang diminati. Ini termasuk dalam rekomendasi berbasis konten.

## install library

In [3]:
# !pip install textsearch
# !pip install contractions
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

In [2]:
import pathlib
pathlib.Path().resolve()

WindowsPath('C:/Users/LENOVO/OneDrive/Documents/STKI/Minggu 4/Similarity')

## load dan view data

In [10]:
import pandas as pd
import os

# Ganti path ini sesuai lokasi file CSV di komputermu
file_path = r"C:/Users/LENOVO/Documents/STKI/Minggu 4/tmdb_5000_movies.csv.gz"

# Cek apakah file ada
if os.path.exists(file_path):
    df = pd.read_csv(file_path, compression='gzip')
    print("File berhasil dibaca!")
    display(df.head())  # menampilkan 5 baris pertama
else:
    print("File tidak ditemukan! Periksa path berikut:", file_path)

File tidak ditemukan! Periksa path berikut: C:/Users/LENOVO/Documents/STKI/Minggu 4/tmdb_5000_movies.csv.gz


In [1]:
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw()
file_path = askopenfilename(title="Pilih file CSV", filetypes=[("CSV files", "*.csv *.csv.gz")])

if file_path:
    df = pd.read_csv(file_path, compression='gzip') if file_path.endswith(".gz") else pd.read_csv(file_path)
    df = df[['title', 'tagline', 'overview', 'popularity']]
    df['tagline'].fillna('', inplace=True)
    df['description'] = df['tagline'].map(str) + ' ' + df['overview']
    df.dropna(inplace=True)
    df = df.sort_values(by=['popularity'], ascending=False)
    df.info()
    display(df.head())
else:
    print("Tidak ada file yang dipilih!")

Tidak ada file yang dipilih!


In [ ]:
df = df[['title', 'tagline', 'overview', 'popularity']]
df.tagline.fillna('', inplace=True)
df['description'] = df['tagline'].map(str) + ' ' + df['overview']
df.dropna(inplace=True)
df = df.sort_values(by=['popularity'], ascending=False)
df.info()

In [6]:
df.head()

,title,tagline,overview,popularity,description
546,Minions,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",875.581305,"Before Gru, they had a history of bad bosses M..."
95,Interstellar,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,724.247784,Mankind was born on Earth. It was never meant ...
788,Deadpool,Witness the beginning of a happy ending,Deadpool tells the origin story of former Spec...,514.569956,Witness the beginning of a happy ending Deadpo...
94,Guardians of the Galaxy,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",481.098624,All heroes start somewhere. Light years from E...
127,Mad Max: Fury Road,What a Lovely Day.,An apocalyptic story set in the furthest reach...,434.278564,What a Lovely Day. An apocalyptic story set in...


In [7]:
df.iloc[0].description

'Before Gru, they had a history of bad bosses Minions Stuart, Kevin and Bob are recruited by Scarlet Overkill, a super-villain who, alongside her inventor husband Herb, hatches a plot to take over the world.'

# Bangun Sistem Rekomendasi Film

Di sini Anda akan membangun sistem rekomendasi film Anda sendiri. Kami akan menggunakan pipa berikut:
- Teks pra-pemrosesan
- Rekayasa Fitur
- Komputasi Kemiripan Dokumen
- Temukan film serupa teratas
- Bangun fungsi rekomendasi film


## Kemiripan Dokumen / document similarity

Rekomendasi adalah tentang memahami fitur dasar yang membuat kita menyukai satu pilihan daripada yang lain. Kesamaan antar item (dalam hal ini film) adalah salah satu cara untuk memahami mengapa kita memilih satu film daripada yang lain. Ada berbagai cara untuk menghitung kesamaan antara dua item. Salah satu ukuran yang paling banyak digunakan adalah __cosine similarity__ yang telah kita gunakan di unit sebelumnya.

### Kemiripan Kosinus

Cosine Similarity digunakan untuk menghitung skor numerik untuk menunjukkan kesamaan antara dua dokumen teks. Secara matematis, ini didefinisikan sebagai berikut:

$$ cosinus(x,y) = \frac{x. y^\intercal}{||x||.||y||} $$

In [8]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['description']))
len(norm_corpus)

4800

## Extrak TF-IDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(4800, 20471)

## Compute Pairwise Document Similarity

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.000000,0.000000,0.000000,0.006071,0.008067,0.0,0.000000,0.0,0.000000,...,0.018758,0.000000,0.037930,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.009646
1,0.000000,1.000000,0.000000,0.017839,0.007968,0.000000,0.0,0.012501,0.0,0.014840,...,0.000000,0.000000,0.017564,0.0,0.019152,0.000000,0.000000,0.000000,0.000000,0.007963
2,0.000000,0.000000,1.000000,0.000000,0.017178,0.000000,0.0,0.000000,0.0,0.024326,...,0.000000,0.006903,0.005024,0.0,0.012893,0.000000,0.025975,0.000000,0.027126,0.009340
3,0.000000,0.017839,0.000000,1.000000,0.000000,0.022414,0.0,0.000000,0.0,0.037207,...,0.000000,0.060846,0.025039,0.0,0.036237,0.030516,0.022605,0.000000,0.000000,0.000000
4,0.006071,0.007968,0.017178,0.000000,1.000000,0.004673,0.0,0.064581,0.0,0.000000,...,0.022064,0.019662,0.036561,0.0,0.015826,0.000000,0.076033,0.004516,0.043475,0.011465


## mendapatkan title / judul movies

In [11]:
movies_list = df['title'].values
movies_list, movies_list.shape

(array(['Minions', 'Interstellar', 'Deadpool', ..., 'Penitentiary',
        'Alien Zone', 'America Is Still the Place'], dtype=object),
 (4800,))

## Temukan Film Serupa Teratas untuk Contoh Film

Mari ambil __Minions__ film paling populer dari kerangka data di atas dan coba temukan film paling mirip yang dapat direkomendasikan

### ambil movie ID

In [12]:
movie_idx = np.where(movies_list == 'Minions')[0][0]
movie_idx

0

### ambil similarities

In [13]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([1.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.00964646])

### Get top 5 similar movie IDs

In [14]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([ 33,  60, 737, 490, 298])

### Get top 5 similar movies

In [ ]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

NameError: name 'df' is not defined

### Buat fungsi rekomendasi film untuk merekomendasikan 5 film serupa teratas untuk film apa pun


The movie title, movie title list and document similarity matrix dataframe akan diberikan sebagai input ke fungsi

In [16]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

# misal cari rekomendasi untuk film film berikut

In [7]:
popular_movies = ['Minions', 'Interstellar', 'Deadpool', 'Jurassic World', 'Pirates of the Caribbean: The Curse of the Black Pearl',
              'Dawn of the Planet of the Apes', 'The Hunger Games: Mockingjay - Part 1', 'Terminator Genisys', 
              'Captain America: Civil War', 'The Dark Knight', 'The Martian', 'Batman v Superman: Dawn of Justice', 
              'Pulp Fiction', 'The Godfather', 'The Shawshank Redemption', 'The Lord of the Rings: The Fellowship of the Ring',  
              'Harry Potter and the Chamber of Secrets', 'Star Wars', 'The Hobbit: The Battle of the Five Armies',
              'Iron Man']

In [8]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

Movie: Minions


NameError: name 'movie_recommender' is not defined

# Rekomendasi Film dengan Penyematan

Kita menggunakan fitur normalisasi berbasis hitungan di bagian sebelumnya. Bisakah kita menggunakan penyematan kata dan kemudian menghitung kesamaan film? kita pasti bisa! Di sini kita akan menggunakan model FastText dan melatihnya di korpus kita.

Model FastText pertama kali diperkenalkan oleh Facebook pada tahun 2016 sebagai perpanjangan dan seharusnya merupakan peningkatan dari model vanilla Word2Vec. Berdasarkan makalah asli berjudul 'Enriching Word Vectors with Subword Information / Memperkaya Vektor Kata dengan Informasi Subword' oleh Mikolov et al. yang merupakan bacaan yang bagus untuk mendapatkan pemahaman mendalam tentang cara kerja model ini. Secara keseluruhan, FastText adalah kerangka kerja untuk mempelajari representasi kata dan juga melakukan klasifikasi teks yang kuat, cepat, dan akurat. Framework ini bersumber terbuka dari Facebook di GitHub dan mengklaim memiliki yang berikut ini.
- Vektor kata bahasa Inggris terkini.
- Vektor kata untuk 157 bahasa dilatih di Wikipedia dan Perayapan.
- Model untuk identifikasi bahasa dan berbagai tugas yang diawasi.

Meskipun saya belum mengimplementasikan model ini dari awal, berdasarkan makalah penelitian, berikut adalah apa yang saya pelajari tentang cara kerja model tersebut. Secara umum, model prediktif seperti model Word2Vec biasanya menganggap setiap kata sebagai entitas yang berbeda (mis. `di mana`) dan menghasilkan penyematan padat untuk kata tersebut. Namun hal ini menjadi batasan serius dengan bahasa yang memiliki kosakata besar dan banyak kata langka yang mungkin tidak banyak muncul di kumpulan yang berbeda. Model Word2Vec biasanya mengabaikan struktur morfologi setiap kata dan menganggap kata sebagai satu kesatuan. Model FastText menganggap setiap kata sebagai Bag of Character n-gram. Ini juga disebut sebagai model subword di koran.

Kami menambahkan simbol batas khusus < dan > di awal dan akhir kata. Hal ini memungkinkan kita untuk membedakan prefiks dan sufiks dari rangkaian karakter lainnya. Kami juga menyertakan kata w itu sendiri dalam himpunan n-gramnya, untuk mempelajari representasi setiap kata (selain karakter n-gramnya). Mengambil kata `di mana` dan n=3 (tri-gram) sebagai contoh, itu akan diwakili oleh karakter n-gram: `<wh, whe, her, ere, re>` dan urutan khusus `<where >` mewakili seluruh kata. Perhatikan bahwa urutan , yang sesuai dengan kata `<her>` berbeda dengan tri-gram `her` dari kata `where`.

Di sini kami memanfaatkan `gensim` untuk membuat embedding kami

In [9]:
from gensim.models import FastText

tokenized_docs = [doc.split() for doc in norm_corpus]
ft_model = FastText(tokenized_docs, window=30, min_count=2, workers=4, sg=1)

ModuleNotFoundError: No module named 'gensim'

# Hasilkan penyematan tingkat dokumen / Word Embedding

Model penyematan kata (word embedding) memberi kita penyematan untuk setiap kata, bagaimana kita dapat menggunakannya untuk tugas ML\DL hilirisasi? salah satu caranya adalah dengan meratakannya atau menggunakan model sekuensial. Pendekatan yang lebih sederhana adalah dengan rata-rata semua penyematan kata untuk kata-kata dalam dokumen dan menghasilkan penyematan tingkat dokumen dengan panjang tetap (fixed-length document level emebdding)

In [11]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [12]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 100)
doc_vecs_ft.shape

NameError: name 'tokenized_docs' is not defined

# Dapatkan Rekomendasi Film

Kami akan memanfaatkan kesamaan kosinus lagi untuk menghasilkan rekomendasi

In [14]:
doc_sim = cosine_similarity(doc_vecs_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

NameError: name 'cosine_similarity' is not defined

In [15]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

Movie: Minions


NameError: name 'movie_recommender' is not defined